In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
batch_size = 100  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 20000  # Number of samples to train on.

In [3]:

column_names = ['ENGLISH', 'FRENCH']

# Read the CSV file with named columns
df = pd.read_csv('/content/fra.csv', header=None, names=column_names, encoding='latin-1')
df_1=df.sample(frac=0.95, random_state=42)


In [4]:
#removing all special characters that are not letters or numbers in our text data ,replacing the removed characters with spaces and keeping spaces between words without removal to distinguish between words
df_1 = df_1.apply(lambda x: x.str.replace(r'\W', ' ') if x.dtype == 'O' else x, axis=0)

<ipython-input-4-09c935406cb1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_1 = df_1.apply(lambda x: x.str.replace(r'\W', ' ') if x.dtype == 'O' else x, axis=0)


In [5]:
df_1 = df_1.applymap(lambda x: x.lower() if isinstance(x, str) else x) # lower_case

In [6]:
#creating function to apply tokenization
def tokenize(column):
    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]

In [7]:
df_1 = df_1.applymap(lambda x: tokenize(x) if isinstance(x, str) else x)


In [8]:

input_column_name = 'ENGLISH'
target_column_name = 'FRENCH'

# Vectorize the data
input_texts = df_1[input_column_name].astype(str).tolist()
target_texts = ['\t' + text + '\n' for text in df_1[target_column_name].astype(str).tolist()]



In [9]:
# Build character sets
input_characters = set(''.join(input_texts))
target_characters = set(''.join(target_texts))

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])   #Finds the length of the longest input sequence among all the input_texts.
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [10]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 44578
Number of unique input tokens: 32
Number of unique output tokens: 45
Max sequence length for inputs: 47
Max sequence length for outputs: 118


In [11]:

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])  #creates a list of tuples where each tuple contains a character and its corresponding index.
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
#encoding text sequences into numerical sequences by replacing characters with their respective indices

In [12]:
# Text Preprocessing
# One-hot encoding for encoder and decoder input data
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
#These arrays serve as placeholders to store the encoded input and target sequences.
# During the training process, the actual encoded sequences will be filled into these arrays to feed into the encoder and decoder of the sequence-to-sequence model

In [13]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):          #For each character in the input_text 
        encoder_input_data[i, t, input_token_index[char]] = 1.  #it retrieves its index input_token_index dictionary 
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.   #sets the corresponding position in the encoder_input_data to 1.  'O-H'
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.   ##it retrieves its index target_token_index dictionary 
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.     # sets the position in decoder_target_data to 1 for the previous character
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.              # the model learns by predicting the next token given the previous one
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [14]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [15]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [16]:
#Train a basic LSTM-based Seq2Seq model to predict decoder_target_data given encoder_input_data and decoder_input_data

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [17]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Epoch 1/100
510/510 [==============================] - 18s 27ms/step - loss: 0.7442 - accuracy: 0.8050 - val_loss: 0.4543 - val_accuracy: 0.8695
Epoch 2/100
510/510 [==============================] - 11s 21ms/step - loss: 0.4359 - accuracy: 0.8731 - val_loss: 0.3968 - val_accuracy: 0.8811
Epoch 3/100
510/510 [==============================] - 11s 21ms/step - loss: 0.3757 - accuracy: 0.8869 - val_loss: 0.3637 - val_accuracy: 0.8907
Epoch 4/100
510/510 [==============================] - 11s 22ms/step - loss: 0.3648 - accuracy: 0.8924 - val_loss: 0.3353 - val_accuracy: 0.8989
Epoch 5/100
510/510 [==============================] - 11s 22ms/step - loss: 0.3274 - accuracy: 0.9005 - val_loss: 0.3218 - val_accuracy: 0.9025
Epoch 6/100
510/510 [==============================] - 12s 24ms/step - loss: 0.3102 - accuracy: 0.9056 - val_loss: 0.3061 - val_accuracy: 0.9062
Epoch 7/100
510/510 [==============================] - 11s 21ms/step - loss: 0.2960 - accuracy: 0.9096 - val_loss: 0.2948 - val_ac

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

In [19]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [23]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [24]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 20ms/step
-
Input sentence: ['how', 'are', 'you', 'getting', 'on']
Decoded sentence: ['comment', 'pourrais', 'je', 'rester']

1/1 [==============================] - 0s 20ms/step
-
Input sentence: ['everyone', 'was', 'fighting']
Decoded sentence: ['tout', 'le', 'monde', 'est', 'en', 'train', 'de', 'démiser']

1/1 [==============================] - 0s 28ms/step
-
Input sentence: ['that', 'could', 'be', 'wrong']
Decoded sentence: ['cette', 'porte', 'est', 'propre']

1/1 [==============================] - 0s 19ms/step
-
Input sentence: ['you', 'can', 't', 'handle', 'it']
Decoded sentence: ['tu', 'ne', 'peux', 'pas', 'me', 'lever']

1/1 [==============================] - 0s 21ms/step
-
Input sentence: ['i', 'let', 'them', 'go']
Decoded sentence: ['j', 'aime', 'les', 'cheveux']

1/1 [==============================] - 0s 19ms/step
-
Input sentence: ['do', 'you', 'want', 'something']
Decoded sentence: ['aimez', 'vous', 'le', 'sac']

1/1 [==============

In [ ]:
from sklearn.model_selection import train_test_split 
 
# Splitting the sample data into training and testing sets 
train_df, test_df = train_test_split(df_1, test_size=0.2, random_state=42) 

In [ ]:
from nltk.translate.bleu_score import corpus_bleu 
from rouge import Rouge 
from nltk.translate.meteor_score import meteor_score 
 
# Assuming you have separate datasets for training and evaluation 
train_input_texts = train_df[input_column_name].astype(str).tolist() 
train_target_texts = ['\t' + text + '\n' for text in train_df[target_column_name].astype(str).tolist()] 
 
eval_input_texts = test_df[input_column_name].astype(str).tolist() 
eval_target_texts = ['\t' + text + '\n' for text in test_df[target_column_name].astype(str).tolist()] 
 
# Function to clean and preprocess the input sequence 
def preprocess_input_sequence(input_seq): 
    # Implement any necessary preprocessing for the input sequence 
    return input_seq 
 
# Function to decode a sequence using the trained model 
def decode_sequence(input_seq): 
    
    return decoded_sentence 
 
# Evaluate BLEU, ROUGE, and METEOR scores for training and evaluation datasets 
def evaluate_dataset(input_texts, target_texts): 
    bleu_scores = [] 
    rouge_scores = [] 
    meteor_scores = [] 
 
    for i in range(len(input_texts)): 
        input_seq = preprocess_input_sequence(input_texts[i]) 
        decoded_sentence = decode_sequence(input_seq) 
 
        # Reference sentences for the current sequence 
        reference = [tokenize(target_texts[i])] 
 
        # BLEU score 
        bleu_score = sentence_bleu(reference, tokenize(decoded_sentence)) 
        bleu_scores.append(bleu_score) 
 
        # ROUGE score 
        rouge = Rouge() 
        rouge_score = rouge.get_scores(decoded_sentence, target_texts[i]) 
        rouge_scores.append(rouge_score) 
 
        # METEOR score 
        meteor_score_value = meteor_score(target_texts[i], decoded_sentence) 
        meteor_scores.append(meteor_score_value) 
 
    return bleu_scores, rouge_scores, meteor_scores 
 
# Evaluate on training dataset 
train_bleu, train_rouge, train_meteor = evaluate_dataset(train_input_texts, train_target_texts) 
 
# Evaluate on evaluation dataset 
eval_bleu, eval_rouge, eval_meteor = evaluate_dataset(eval_input_texts, eval_target_texts) 
 
# Print or use the evaluation scores as needed 
print(f'Training BLEU Score: {sum(train_bleu) / len(train_bleu)}') 
print(f'Training ROUGE Scores: {sum(train_rouge) / len(train_rouge)}') 
print(f'Training METEOR Score: {sum(train_meteor) / len(train_meteor)}') 
 
print(f'Evaluation BLEU Score: {sum(eval_bleu) / len(eval_bleu)}') 
print(f'Evaluation ROUGE Scores: {sum(eval_rouge) / len(eval_rouge)}') 
print(f'Evaluation METEOR Score: {sum(eval_meteor) / len(eval_meteor)}')